In [ ]:
# Introduction to Python (CIST 005A) Midterm Project
# First-grade Math Electronic Flash Cards: Math1Flash
# Author: Steve DeGrange
#
# Math1Flash is a simulation of a flash card educational activity.
#
# With physical flash cards, you have a stack of cards with a problem
# on the front side and the answer on the reverse side. A card is
# drawn and you are shown the front side for you to answer. If
# you don't get the answer right, you are shown the reverse side.
#
# In Math1Flash, you (supposedly a first-grader learning math) are
# given a random arithmetic or number compare problem to answer
# (a card is drawn from the stack of shuffled cards) and shown the
# problem (the front side). You are given up to three chances to
# answer correctly and then you are shown the answer (reverse side).
#
# Examples: What is 9 x 11?  answer a number     (arithmetic)
#           Is 10 > 16?      answer yes or no    (compare numbers)
#
# If you answer correctly, you are rewarded with praise.
#
# If you don't answer correctly, you are given up to two more
# tries (three total) to get it right and then shown the answer.
# Before each retry you're given a "hint" such as "That's too big"
# or "That's not yes or no".
#
# You can skip a problem/retry by just entering "" instead of an
# answer if you want to go to another question instead (or end).
#
# This is repeated (like drawing another card from the stack)
# until you reply negatively to wanting another problem question.
#
# In addition to entering "" to skip a problem, you can abbreviate
# "yes" and "no" down to one letter at all yes/no prompts and take
# the default of "yes" at a yes/no continuation-type prompt.
#
# Emojis and message coloring are used to make it more visually stimulating:
#      Arithmetic items are in blue.
#      Compare numbers items are in green.
#      Kudos output is partly in red.
#      Emojis strewn throughout

import random                                 # for randint and choice

# Define constants
RED, GREEN, BLUE, BOLD, END = "\033[0;31m\033[1m","\033[0;32m\033[1m","\033[0;34m\033[1m", "\033[1m", "\033[0m"  # ANSI codes
ARITHCOLOR, COMPCOLOR = BLUE, GREEN           # for coloring some output specific to the type of operator

yesNo = ("yes", "ye", "y", "no", "n")         # Valid replies to yes/no prompts

arithmetic, compare = ("+", "-", "x"), ("=", "≠","<", ">", "≤", "≥")   # Supported operators

SALUTE, GRIN1, GRIN2, PARTY1, PARTY2 = "\U0001fae1", "\U0001f600", "\U0001f601", "\U0001f973", "\U0001f389"      # Emojis
XFINGERS, TROPHY, MEDAL, SAD, WORRY = "\U0001f91e", "\U0001f3c6", "\U0001f947", "\U0001f61e", "\U0001f61f"       # Emojis
HAND, THUMBUP, THUMBDOWN = "\U0000270b", "\U0001f44d", "\U0001f44e"                                              # Emojis

# Define global variables
student, attempts, correct = "", 0, 0

### Function getOperator returns a randomly chosen allowed arithmetic or compare operator
def getOperator():
    operator = random.choice(arithmetic * 2 + compare)   # double up on arithmetic since compare has 2x more ops
    opType = "arithmetic" if operator in arithmetic else "comparing"
    opColor = ARITHCOLOR if operator in arithmetic else COMPCOLOR
    return operator, opType, opColor

### Function getProblem returns the operands, question and its answer given the operator
def getProblem(operator):
    def getOperands(operator):           # Subfunction getOperands returns a list of two random whole numbers for operands
        maxOperand = 15 if operator == "x" else 25    # cut back for multiplication
        return [random.randint(0, maxOperand), random.randint(0, maxOperand)]
    def getQuestion(operator, operands): # Subfunction getQuestion returns the current problem as a question, such as "What is 18 x 3?".
        question = ["", "Is ", "%d %s %d" % (operands[0], operator, operands[1]), " ?   (answer yes or no, or Enter to skip)"]  # init as compare
        if operator in arithmetic:       # change form of question from "Is xxx? to "What is xxx?"
            question[1] = "What is "
            question[3] = " ?   (answer a number, or Enter to skip)"
        question[0] = question[1] + question[2] + question[3]
        return question
    def getAnswer(operator, operands):   # Subfunction getAnswer returns the answer to the current problem, such as 54 for 18 x 3.
        answer = None
        match operator:
            case "+":
                answer = operands[0] + operands[1]
            case "-":
                answer = operands[0] - operands[1]
            case "x":
                answer = operands[0] * operands[1]
            case "=":
                answer = "yes" if operands[0] == operands[1] else "no"
            case "≠":
                answer = "yes" if operands[0] != operands[1] else "no"
            case "<":
                answer = "yes" if operands[0] < operands[1] else "no"
            case ">":
                answer = "yes" if operands[0] > operands[1] else "no"
            case "≤":
                answer = "yes" if operands[0] <= operands[1] else "no"
            case "≥":
                answer = "yes" if operands[0] >= operands[1] else "no"
        return answer
    operands = getOperands(operator)
    return operands, getQuestion(operator, operands), getAnswer(operator, operands)

### Function getHint returns text for hinting why an incorrect answer was wrong
def getHint(answerStudent, answerCorrect, operator):
    if operator in compare:
        return "not yes or no" if answerStudent not in yesNo else "wrong, but you'll get it"
    if not answerStudent.isdigit():
        return "not a whole number"
    return "not big enough" if int(answerStudent) < answerCorrect else "too big"

### Function getYesNo asks for "yes" or "no" (or "" for default) or a shortening like "n" or "ye" and returns as True/False
def getYesNo(prompt, default = "yes"):
    reply = input(prompt).strip().lower()
    if reply == "":
        reply = default
    while reply not in yesNo:
        reply = input("Please say yes or no. Try again.\n ").strip().lower()
    return True if reply[0] == "y" else False

### Function kudos returns a praising phrase for a correct answer the a given try, or a supportive phrase for an incorrect one
def kudos(answerTry, answer, opColor):
    match answerTry:
        case 0:  # they didn't get it right after three attempts so kudos in an encouragement sense only
            print(f"\n{THUMBDOWN} That's not right, either. It was hard.\nThe answer was {opColor}{answer}{END}.")
        case 1:  # success on the first attempt for this problem; pick a random kudos praising phrase so its not too boring
            match random.randint(0, 4):
                case 0:
                    print(f"\n{GRIN1} {RED}Right!{END} You're really super at this {BOLD}{student}{END}!")
                case 1:
                    print(f"\n{PARTY1} {RED}Correct!{END} You're doing great, {BOLD}{student}{END}!")
                case 2:
                    print(f"\n{GRIN1} {RED}You got it, {student}!{END} You're doing really good{END}.")
                case 3:
                    print(f"\n{PARTY2} {RED}Nice job, {student}!{END} You're super at this!")
                case _:
                    print(f"\n{GRIN2} {RED}That's right!{END} Keep it up, {BOLD}{student}.{END}")
        case 2:  # success on the second attempt for this problem              
            print(f"\n{GRIN2} {RED}Correct!{END} You got it, {BOLD}{student}{END}! {THUMBUP}")
        case 3:  # success on the third attempt for this problem
            print(f"{GRIN1} {RED}Yes, you did it!!{END} Good job, {BOLD}{student}{END}! {THUMBUP}")
    return

### Function validateAnswer returns True if the answer given was correct and False if incorrect and prints kudos messages
def validateAnswer(tries, answerStudent, answerCorrect, operator, color):
    global student, attempts, correct
    validAnswer = False
    if answerStudent != "":
        if operator in arithmetic:
            validAnswer = True if answerStudent.isdigit() and int(answerStudent) == answerCorrect else False
        else:
            validAnswer = True if answerCorrect.startswith(answerStudent) else False
    if validAnswer:
        correct += 1
        kudos(tries, answerCorrect, color)
    elif tries == 3:
        kudos(0, answerCorrect, color)
    return validAnswer

### Fuction initialize gets the student name and returns if they are ready to start
def initialize():
    global student
    student = input("My name is Flash. What's yours? ")
    if student == "":
        student = "math student"
    print(f"I'm pleased to meet you, {BOLD}{student}{END}! {HAND}\n")
    print(f"It's time for {BOLD}math{END}!")
    print(f"Some {ARITHCOLOR}arithmetic{END} and some {COMPCOLOR}comparing{END}.")
    return getYesNo(f"\nAre you ready? {THUMBUP} {THUMBDOWN}\n   Say yes or no.   (or Enter to start)\n  ")
    
### Function terminate prints end messages
def terminate():
    global student, attempts, correct
    if attempts > 1 and correct > attempts / 2:  # More than half right?
        if correct == attempts:                  # Got all of them right?
            print(f"{RED}You didn't miss any! {correct} out of {attempts}!{END} Most excellent!{TROPHY}")
        else:
            print(f"{RED}You got {correct} out of {attempts} right!{END} Nice job! {MEDAL}")
    elif attempts > 0:
        print(f"You got {correct} out of {attempts} right.")
        
    print(f"Bye, {BOLD}{student}{END}. Come back soon. {HAND}")

### main code
def main():
    global student, attempts, correct
    moreMath = initialize()
    while moreMath:
        # Build a new question and answer
        operator, opType, opColor = getOperator()                        # get a new operator and operator attributes
        operands, question, answer = getProblem(operator)                # generate operands, question and answer
        while (operator == 'x' and answer > 100) or (operator == '-' and answer < 0):
            operands, question, answer = getProblem(operator)            # regenerate operands, question and answer
        # Present the question and ask for the answer, repeating up to twice; a reply of "" means skip the problem
        print(f"\nTry this {opColor}{opType}{END} problem:\n   {question[1]}{opColor}{question[2]}{END}{question[3]}")
        reply = input("   ").strip().lower()
        if reply != "":                                                  # if they didn't skip the question and don't count
            attempts += 1
            if not validateAnswer(1, reply, answer, operator, opColor):  # if they got it wrong on 1st try, try again
                reply = input(f"\n{WORRY} That's {getHint(reply, answer, operator)}. Try again! {XFINGERS}\n   ").strip().lower()
                if reply != "":                                          # "" on 2nd try skips the question but counts as a try
                    if not validateAnswer(2, reply, answer, operator, opColor):  # if they got it wrong on 2nd try, try one last time
                        reply = input(f"\n{SAD} That's {getHint(reply, answer, operator)}. Try once more. {XFINGERS}\n   ").strip().lower()
                        if reply != "":                                  # "" on 3rd try skips the question but counts as a try
                            validateAnswer(3, reply, answer, operator, opColor)  # last chance at answering
        if reply == "":                                                  # Problem was skipped. Tell them the correct answer.
            print(f"OK. The answer was {opColor}{answer}{END}.")      # See if they want to get a new math problem or stop
        moreMath = getYesNo(f"\nDo you want a new problem? {THUMBUP} {THUMBDOWN}\n   Say yes or no.   (or Enter for another)\n  ")
    terminate()
    
# Run main code
if __name__ == "__main__":
    main()
